<h1><center>Введение в анализ данных</center></h1>
<hr>
<h2><center>Quora Question Pairs</center></h2>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)

<h2>Подготовка данных:</h2>

<h3>Основная таблица</h3>

In [3]:
#filepath = '/media/ivan/Acer/Ivan/sphere/Quora/train.csv'
filepath = 'C:\Ivan\sphere\Quora\\train.csv'
df = pd.read_csv(filepath, sep=',')
df = df.reset_index(drop=True)
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


Доля дублирующихся вопросов

In [3]:
df.is_duplicate.mean()

0.369197853026293

In [8]:
df.shape

(404290, 6)

<h2>Формат выходных данных</h2>

In [5]:
#filepath = '/media/ivan/Acer/Ivan/sphere/Quora/sample_submission.csv'
filepath = 'C:\Ivan\sphere\Quora\\sample_submission.csv'
df_out = pd.read_csv(filepath, sep=',')
print len(df_out)
df_out.head()

2345796


,test_id,is_duplicate
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1


<h2>Shallow benchmark</h2>

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [4]:
df = df.dropna(how='any')
df.shape

(404288, 6)

Удалили всего две строчки с пропущенными значениями

Построим Bag of Words на всей выборке вопросов

In [5]:
BOW = CountVectorizer(max_df=0.999, min_df=1000, max_features=1000, 
                                      analyzer='char', ngram_range=(1,3), 
                                      binary=True, lowercase=True, stop_words=None)

In [6]:
%%time
BOW.fit(pd.concat((df.loc[:,'question1'],df.loc[:,'question2'])).unique())

Wall time: 1min 31s


CountVectorizer(analyzer='char', binary=True, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.999, max_features=1000, min_df=1000,
        ngram_range=(1, 3), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [7]:
%%time
BOW_1 = BOW.transform(df.loc[:,'question1'])
BOW_2 = BOW.transform(df.loc[:,'question2'])
labels = df.is_duplicate.values

Wall time: 2min 30s


In [8]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import scale

from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
import time

In [9]:
X = -(BOW_1 != BOW_2).astype(int)
y = labels
X.shape, y.shape

((404288, 1000), (404288L,))

In [10]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score

def score_on_BOW(bag_1, bag_2):
    clf = LogisticRegression(C=0.1, solver='sag')
    X = -(bag_1 != bag_2).astype(int)
    y=df.is_duplicate.values
    print X.shape, y.shape
    res = cross_val_score(clf, X, y, scoring='neg_log_loss', cv=10)
    print res.mean(), res.std()

In [11]:
score_on_BOW(BOW_1, BOW_2)

(404288, 1000) (404288L,)
-0.489694075717 0.00166022701331


In [ ]:
%%time
model = LogisticRegression(C=0.1, solver='sag', class_weight={1: 0.46, 0: 1.32}).fit(X, y)

Wall time: 26.3 s


In [ ]:
%%time
filepath = 'C:\Ivan\sphere\Quora\\test.csv'
df = pd.read_csv(filepath)
df.loc[df['question1'].isnull(),['question1','question2']] = 'random empty question'
df.loc[df['question2'].isnull(),['question1','question2']] = 'random empty question'

BOW_1 = BOW.transform(df.loc[:,'question1'])
BOW_2 = BOW.transform(df.loc[:,'question2'])

X = -(BOW_1 != BOW_2).astype(int)

Wall time: 35min


In [ ]:
X.shape

(2345796, 1000)

In [ ]:
X_test = scale(X_test)

In [ ]:
%%time
scores = model.predict_proba(X)

Wall time: 4.68 s


In [ ]:
#filepath = '/media/ivan/Acer/Ivan/sphere/Quora/quora.csv'
filepath = 'C:\Ivan\sphere\Quora\\quora.csv'

submission = pd.DataFrame()
submission['test_id'] = df['test_id']
submission['is_duplicate'] = scores[:,1]
submission.to_csv(filepath, index=False)

In [1]:
#0.38796

<h2>Words</h2>

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [4]:
df = df.dropna(how='any')
df.shape

(404288, 6)

In [5]:
BOW = CountVectorizer(max_df=0.999, min_df=1000, max_features=1000, 
                                      analyzer='word', ngram_range=(1,2), 
                                      binary=True, lowercase=True, stop_words='english')

In [6]:
%%time
BOW.fit(pd.concat((df.loc[:,'question1'],df.loc[:,'question2'])).unique())

Wall time: 31.3 s


CountVectorizer(analyzer='word', binary=True, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.999, max_features=1000, min_df=1000,
        ngram_range=(1, 2), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [7]:
%%time
BOW_1 = BOW.transform(df.loc[:,'question1'])
BOW_2 = BOW.transform(df.loc[:,'question2'])
labels = df.is_duplicate.values

Wall time: 36 s


In [8]:
X = -(BOW_1 != BOW_2).astype(int)
y = labels
X.shape, y.shape

((404288, 467), (404288L,))

In [9]:
from sklearn.linear_model import LogisticRegression

In [10]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score

def score_on_BOW(bag_1, bag_2):
    clf = LogisticRegression(C=0.1, solver='sag')
    X = -(bag_1 != bag_2).astype(int)
    y=df.is_duplicate.values
    print X.shape, y.shape
    res = cross_val_score(clf, X, y, scoring='neg_log_loss', cv=10)
    print res.mean(), res.std()

In [11]:
score_on_BOW(BOW_1, BOW_2)

(404288, 467) (404288L,)
-0.584828358131 0.00110979584299


In [10]:
%%time
model = LogisticRegression(C=0.1, solver='sag', class_weight={1: 0.46, 0: 1.32}).fit(X, y)

Wall time: 5.38 s


In [11]:
%%time
filepath = 'C:\Ivan\sphere\Quora\\test.csv'
testDF = pd.read_csv(filepath)
testDF.ix[testDF['question1'].isnull(),['question1','question2']] = 'random empty question'
testDF.ix[testDF['question2'].isnull(),['question1','question2']] = 'random empty question'

testQuestion1_BOW_rep = BOW.transform(testDF.ix[:,'question1'])
testQuestion2_BOW_rep = BOW.transform(testDF.ix[:,'question2'])

X_test = -(testQuestion1_BOW_rep != testQuestion2_BOW_rep).astype(int)

Wall time: 5min 53s


In [12]:
%%time
scores = model.predict_proba(X_test)

Wall time: 256 ms


In [13]:
#filepath = '/media/ivan/Acer/Ivan/sphere/Quora/quora.csv'
filepath = 'C:\Ivan\sphere\Quora\\quora.csv'

submission = pd.DataFrame()
submission['test_id'] = testDF['test_id']
submission['is_duplicate'] = scores[:,1]
submission.to_csv(filepath, index=False)

<h2>Cosine similarity</h2>

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics.pairwise import paired_cosine_distances
from sklearn.decomposition import TruncatedSVD

In [5]:
df = df.dropna(how='any')
df.shape

(404288, 6)

In [6]:
BOW = CountVectorizer(max_df=0.999, min_df=1000, max_features=None, 
                                      analyzer='char', ngram_range=(1,3), 
                                      binary=True, lowercase=True)

In [7]:
%%time
BOW.fit(pd.concat((df.loc[:,'question1'],df.loc[:,'question2'])).unique())

Wall time: 1min 28s


CountVectorizer(analyzer='char', binary=True, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.999, max_features=None, min_df=1000,
        ngram_range=(1, 3), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [8]:
%%time
BOW_1 = BOW.transform(df.loc[:,'question1'])
BOW_2 = BOW.transform(df.loc[:,'question2'])
labels = df.is_duplicate.values

Wall time: 2min 21s


In [9]:
BOW_1.shape, BOW_2.shape

((404288, 3556), (404288, 3556))

In [17]:
%%time
df['cosine_similarity_queries'] = paired_cosine_distances(BOW_1, BOW_2)

Wall time: 1min 26s


In [10]:
filepath = 'C:\Ivan\sphere\Quora\\'
df.to_csv(filepath+'data_cos_sim.csv', index=False)

In [24]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score

def score_on_features(data, features):
    clf = LogisticRegression()
    if len(features) == 0:
        X = data[features].values.reshape(-1, 1)
    else:
        X = data[features].values
    y=data.is_duplicate.values
    print X.shape, y.shape
    res = cross_val_score(clf, X, y, scoring='neg_log_loss', cv=10)
    print res.mean(), res.std()

In [11]:
score_on_features(df, ['cosine_similarity_queries'])

(404288L, 1L) (404288L,)
-0.575078997474 0.00117564529615


<h2>TSNE</h2>

In [10]:
from scipy.sparse import hstack, vstack
from sklearn.decomposition import TruncatedSVD

In [11]:
new_matr = vstack([BOW_1, BOW_2])
new_matr.shape

(808576, 3556)

In [12]:
tsvd = TruncatedSVD(n_components=3)

In [13]:
%%time
new_svd = tsvd.fit_transform(new_matr)

Wall time: 1min 59s


In [14]:
new_svd.shape

(808576L, 3L)

In [15]:
q1_new_svd, q2_new_svd = new_svd[:BOW_1.shape[0], :], new_svd[BOW_1.shape[0]:, :]

In [16]:
%%time
features_2 = []
for num_comp in xrange(q1_new_svd.shape[1]):
    f_name = 'comp_svd_diff_{}'.format(num_comp)
    features_2.append(f_name)
    df[f_name] = np.abs(q1_new_svd[:, num_comp] - q2_new_svd[:, num_comp])

Wall time: 14.1 s


In [19]:
score_on_features(df, features_2)

(404288L, 3L) (404288L,)
-0.633412911081 0.00110472653285


In [20]:
score_on_features(df, features_2 + ['cosine_similarity_queries'])

(404288L, 4L) (404288L,)
-0.568401979185 0.00112920450191


In [18]:
tsvd = TruncatedSVD(n_components=6)
new_matr = hstack([BOW_1, BOW_2])
new_matr.shape

(404288, 7112)

In [19]:
filepath = 'C:\Ivan\sphere\Quora\\'
df.to_csv(filepath+'data_cos_sim+vstack.csv', index=False)

In [20]:
%%time
new_svd = tsvd.fit_transform(new_matr)

Wall time: 8min 20s


In [21]:
%%time
features_3 = []
for num_comp in xrange(new_svd.shape[1]):
    f_name = 'svd_paired_feat_{}'.format(num_comp)
    features_3.append(f_name)
    df[f_name] = new_svd[:, num_comp]

Wall time: 10.5 s
Compiler : 228 ms
Parser   : 684 ms


In [22]:
df.shape

(404288, 16)

In [23]:
filepath = 'C:\Ivan\sphere\Quora\\'
df.to_csv(filepath+'data_all_svd.csv', index=False)

In [20]:
df.shape

(404288, 15)

In [25]:
features_1 = ['cosine_similarity_queries']

In [26]:
score_on_features(df, features_3 + features_2 + features_1)

(404288L, 10L) (404288L,)
-0.560326419896 0.00124412377024


In [26]:
score_on_features(df, features_3 + features_2)

(404288L, 8L) (404288L,)
-0.623457645619 0.00144721300323


In [27]:
score_on_features(df, features_3)

(404288L, 5L) (404288L,)
-0.642600994196 0.000938158900899


In [28]:
score_on_features(df, features_3 + features_1)

(404288L, 6L) (404288L,)
-0.563245563725 0.00139663971793


In [27]:
X = df[features_3 + features_2 + features_1].values
y=df.is_duplicate.values
X.shape, y.shape

((404288L, 10L), (404288L,))

In [28]:
%%time
model = LogisticRegression(C=0.1, solver='sag', class_weight={1: 0.46, 0: 1.32}).fit(X, y)

Wall time: 4.4 s


In [30]:
import pickle
#filepath = '/media/ivan/Acer/Ivan/sphere/Quora/models/'
filepath = 'C:\Ivan\sphere\Quora\\'
f = open(filepath + 'log_model1.pickle', 'wb')
pickle.dump(model, f)

In [ ]:
%%time
filepath = 'C:\Ivan\sphere\Quora\\test.csv'
df = pd.read_csv(filepath)
df.loc[df['question1'].isnull(),['question1','question2']] = 'random empty question'
df.loc[df['question2'].isnull(),['question1','question2']] = 'random empty question'

BOW_1 = BOW.transform(df.loc[:,'question1'])
BOW_2 = BOW.transform(df.loc[:,'question2'])

Wall time: 35min 18s


In [ ]:
%%time
df['cosine_similarity_queries'] = paired_cosine_distances(BOW_1, BOW_2)

In [ ]:
tsvd = TruncatedSVD(n_components=3)
new_matr = vstack([BOW_1, BOW_2])
new_matr.shape

In [ ]:
%%time
new_svd = tsvd.fit_transform(new_matr)

In [ ]:
q1_new_svd, q2_new_svd = new_svd[:BOW_1.shape[0], :], new_svd[BOW_1.shape[0]:, :]

In [ ]:
features_2 = []
for num_comp in xrange(q1_new_svd.shape[1]):
    f_name = 'comp_svd_diff_{}'.format(num_comp)
    features_2.append(f_name)
    df[f_name] = np.abs(q1_new_svd[:, num_comp] - q2_new_svd[:, num_comp])

In [ ]:
tsvd = TruncatedSVD(n_components=6)
new_matr = hstack([BOW_1, BOW_2])

In [ ]:
%%time
new_svd = tsvd.fit_transform(new_matr)

In [ ]:
features_3 = []
for num_comp in xrange(new_svd.shape[1]):
    f_name = 'svd_paired_feat_{}'.format(num_comp)
    features_3.append(f_name)
    df[f_name] = new_svd[:, num_comp]

In [ ]:
X = df[features_3 + features_2 + features_1].values

In [ ]:
%%time
scores = model.predict_proba(X)

In [ ]:
#filepath = '/media/ivan/Acer/Ivan/sphere/Quora/quora.csv'
filepath = 'C:\Ivan\sphere\Quora\\quora.csv'

submission = pd.DataFrame()
submission['test_id'] = testDF['test_id']
submission['is_duplicate'] = scores[:,1]
submission.to_csv(filepath, index=False)

<h2>Усреднение моделей</h2>

In [6]:
#filepath = '/media/ivan/Acer/Ivan/sphere/Quora/'
filepath = 'C:\Ivan\sphere\Quora\\'
df1 = pd.read_csv(filepath+'quora_mf1000_char_1-3.csv')
df2 = pd.read_csv(filepath+'quora_mf1000_word_1-2.csv')
#df3 = pd.read_csv(filepath+'hearthstone_tree_ratio3.txt', header=None)
#df4 = pd.read_csv(filepath+'hearthstone_tree_ratio4.txt', header=None)

df1['is_duplicate'] = (df1['is_duplicate'] + df2['is_duplicate'])/2
df1.to_csv(filepath+'quora.csv', index=False)
df1.head()

,test_id,is_duplicate
0,0,0.025940
1,1,0.168860
2,2,0.206964
3,3,0.120346
4,4,0.223580


<h2>Результаты</h2>
* char (1, 3), 1000 features<br>0.36305<br><br>
* усреднение char (1, 3) и word (1, 2), 1000 features<br>0.36336<br><br>
* char (1, 2), 1000 features<br>0.36967<br><br>
* char (1, 2), 300 features<br>0.38796<br><br>
* word (1, 2), 1000 features<br>0.39206<br><br>
* word (1, 2), 300 features<br>0.41075<br><br>
* word (1, 3), 300 features<br>0.41128<br><br>